In [1]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import random

from auxiliaries import *

random.seed(34)

In [2]:
# types of routes
types_berlin = {
    0: "tram",
    1: "subway",
    2: "rail",
    3: "bus",
    4: "ferry",
    5: "walking",
}

## Performing experiments on Berlin network

### Instructions / how the code works:

Step1
- reads in both networks (combined and walking)
- Then converts the walking dataframe to comform with the combined dataframe
- Then creates a new dataframe with the combined and walking dataframes

Step2
- Converts the full network to a graph
- Adds the positions of the node (in x and y coordinates) to the graph

Step3 (fixing the walking graph)
- Since the walking map is not connected, seperately convert only the walking dataframe to a graph
- Then the function "make_walking_connected", takes both G and G_walk, and makes the "walk" part of G connected by adding the edges to G that are necessary to make G_walk connected. It also adds some edges to G_walk because there are stops (nodes) in G that are not in G_walk for some reason

Step4 (experiments)
- The function "percolation_experiments" performs the experiments (see source code in [auxiliaries.py](auxiliaries.py))
- By default the method is "random", but it can also be "degree", "betweenness", "closeness" or "eigenvector". If a method is specified it will then remove the edges in order of the method, otherwise it will remove random edges.
It takes parameters for:
  - G (the graph)
  - p_min (the minimum percolation degree)
  - p_max (the maximum percolation degree)
  - steps (the step size in varying the percolation degree)
  - n_tests (the number of different samples of nodes to test with)
  - n_percolations (the number of different percolations to perform) NOTE: only relevant for random percolation, with the other methods the percolation will be the same every time. If you are using a specific method, just set this to 1.
  - method (the method to use for percolation, default is random)
  - verbose (if you want to see the progress, since the function is quite slow. Default is False)
- The function returns a dataframe with the results of the experiments, and the results can be plotted with the plotting functions at the end of the notebook


In [3]:
# STEP 1
# read in public transport network of berlin and walking network
berlin, berlin_nodes = read_in_network("berlin", "combined")
berlin_walking, nodes_t = read_in_network("berlin", "walk")

# adapt berlin_walking dataframe to match berlin dataframe
berlin_walking = berlin_walking.rename(columns={"d_walk": "duration_avg"})
berlin_walking["duration"] = berlin_walking["duration_avg"].apply(distance_to_duration)
berlin_walking["n_vehicles"] = 0
berlin_walking["route_I_counts"] = 0
berlin_walking["route_type"] = 5

# add berlin_walking to berlin
berlin_full = pd.concat([berlin, berlin_walking], ignore_index=True)

In [4]:
berlin_nodes

stop_I        lat        lon  \
0        105  52.528318  13.320260   
1        106  52.527903  13.323637   
2        107  52.529103  13.315981   
3        108  52.525756  13.309840   
4        109  52.525797  13.314261   
...      ...        ...        ...   
4596   10938  52.769962  13.454593   
4597   10939  52.611806  13.594948   
4598   10940  52.606930  13.601930   
4599   10946  52.734171  13.666572   
4600   12674  52.397446  13.530343   

                                               name  
0                     Wiebestr./Huttenstr. (Berlin)  
1                             Reuchlinstr. (Berlin)  
2                               Neues Ufer (Berlin)  
3                         Ilsenburger Str. (Berlin)  
4                           Goslarer Platz (Berlin)  
...                                             ...  
4596                      Stolzenhagen, Stolzenfels  
4597                        Blumberg (BAR), Gutshof  
4598                     Blumberg (BAR), Liebigstr.  
4599                               Danewitz, Kirche  
4600  Schönefeld (bei Berlin), Bohnsdorfer Chaussee  

[4601 rows x 4 columns]

In [5]:
berlin_full

from_stop_I  to_stop_I     d  duration_avg  n_vehicles  \
0            10924      10920   676         120.0          17   
1            10924      10794   713          60.0           2   
2            10924      10435  1033         132.5          72   
3            10924      10436   696          60.0          53   
4            10925      10492   374         120.0          13   
...            ...        ...   ...           ...         ...   
30199         8144       8150   670         755.0           0   
30200         8145       8150   159         169.0           0   
30201         8147       8151   152         162.0           0   
30202        10910      10911   494         494.0           0   
30203        10921      10922    93         103.0           0   

            route_I_counts  route_type    duration  
0                   720:17           3         NaN  
1                    592:2           3         NaN  
2      720:17,581:54,597:1           3         NaN  
3                   581:53           3         NaN  
4             549:3,550:10           3         NaN  
...                    ...         ...         ...  
30199                    0           5  580.769231  
30200                    0           5  130.000000  
30201                    0           5  124.615385  
30202                    0           5  380.000000  
30203                    0           5   79.230769  

[30204 rows x 8 columns]

In [6]:
# STEP 2
# convert to graph
G = convert_to_graph(berlin_full)
pos = add_positions(G, berlin_nodes)

# STEP 3
# convert walking network to seperate graph
G_walk = convert_to_graph(berlin_walking)

# make G connected by adding walking edges
make_walking_connected(G, G_walk)

{105: (6896221.685034783, 1482804.560227602), 106: (6896145.748634766, 1483180.4861479588), 107: (6896365.325729658, 1482328.224126564), 108: (6895752.903584518, 1481644.6111336977), 109: (6895760.405336229, 1482136.7546024262), 110: (6898066.519585421, 1487335.59746073), 111: (6897661.501765876, 1485429.6964591239), 112: (6898149.795097052, 1487630.9280697636), 113: (6896535.319117556, 1488204.1121277784), 114: (6895755.099218438, 1487810.486408388), 115: (6897763.440614098, 1487364.4292088416), 116: (6896000.282113222, 1483937.347365757), 117: (6896043.281376913, 1483749.440065324), 118: (6896686.651025436, 1483723.6139434637), 119: (6897498.804918813, 1486827.8692632925), 120: (6896937.901223832, 1485160.4146109323), 121: (6895864.698707346, 1486272.2736848209), 122: (6896979.8075202005, 1486174.980449881), 123: (6897505.210244195, 1487010.0992696958), 124: (6895844.022896385, 1485615.8226477043), 125: (6895923.432938384, 1486650.6486339744), 126: (6896381.611269847, 1485448.6207725

#### Perform percolation experiment

In [7]:
# STEP 4
results = percolation_experiments(G, 0, 1, 20, 1, 5, verbose=True)

Starting experiments...

p: 0.0, test: 0, time: 1956.2229130415947, perc: 0
p: 0.0, test: 0, time: 1956.2229130415947, perc: 1
p: 0.0, test: 0, time: 1956.2229130415947, perc: 2
p: 0.0, test: 0, time: 1956.2229130415947, perc: 3
p: 0.0, test: 0, time: 1956.2229130415947, perc: 4
p: 0.05263157894736842, test: 0, time: 1990.5047073562025, perc: 0
p: 0.05263157894736842, test: 0, time: 1984.4086768176294, perc: 1
p: 0.05263157894736842, test: 0, time: 1973.487092849854, perc: 2
p: 0.05263157894736842, test: 0, time: 1975.976811460243, perc: 3
p: 0.05263157894736842, test: 0, time: 1981.9269031426804, perc: 4
p: 0.10526315789473684, test: 0, time: 1998.6535388209568, perc: 0
p: 0.10526315789473684, test: 0, time: 2016.2793168471158, perc: 1
p: 0.10526315789473684, test: 0, time: 1999.5016335810128, perc: 2
p: 0.10526315789473684, test: 0, time: 2012.6996799881347, perc: 3
p: 0.10526315789473684, test: 0, time: 1992.2657030076227, perc: 4
p: 0.15789473684210525, test: 0, time: 2042.66491349

In [ ]:
results_betweenness = percolation_experiments(G, 0, 1, 20, 1, 1, method="degree", verbose=True)

Getting nodes...

Starting experiments...

p: 0.0, test: 0, time: 2114.00732073156, perc: 0


In [ ]:
# dump results to pickle file
import pickle

results.to_pickle("results/results_random.pkl")
results_betweenness.to_pickle("results/results_degree.pkl")

#### Plot results

In [ ]:
# plot two plots next to each other
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
# plot first plot
x = results.groupby("p").mean().reset_index()["p"]
# divide y by 60 to get minutes
y = results.groupby("p").mean().reset_index()["result"]/60
# set x and y label
ax1.set_xlabel("percolation probability (p)")
ax1.set_ylabel("average travel time (minutes)")
ax1.set_title("Random Percolation robustness Berlin")
ax1.plot(x, y)

# plot second plot
x = results_betweenness.groupby("p").mean().reset_index()["p"]
# divide y by 60 to get minutes
y = results_betweenness.groupby("p").mean().reset_index()["result"]/60
# set x and y label
ax2.set_xlabel("percolation probability (p)")
ax2.set_ylabel("average travel time (minutes)")
ax2.set_title("Degree targeted Percolation robustness Berlin")
ax2.plot(x, y)


In [ ]:
# plot both lines on the same graph
fig, ax = plt.subplots(figsize=(10, 5))

# plot first line
x1 = results.groupby("p").mean().reset_index()["p"]
y1 = results.groupby("p").mean().reset_index()["result"] / 60
ax.plot(x1, y1, label="Random Percolation")

# plot second line
x2 = results_betweenness.groupby("p").mean().reset_index()["p"]
y2 = results_betweenness.groupby("p").mean().reset_index()["result"] / 60
ax.plot(x2, y2, label="Degree Targeted Percolation")

# set x and y labels
ax.set_xlabel("percolation probability (p)")
ax.set_ylabel("average travel time (minutes)")

# set title
ax.set_title("Percolation Robustness Comparison in Berlin")

# add legend
ax.legend()

# display the plot
plt.show()